In [9]:
import pandas as pd
import json
import os
print("Combingin all data sources.........")

Combingin all data sources.........


In [10]:
all_conversations = []

In [11]:
# Survery form data
# Open your JSONL file properly
with open("data/processed/survey.jsonl", "r", encoding="utf-8") as f:
    for line in f:
        line = line.strip()
        if not line:
            continue
        try:
            data = json.loads(line)
            # Make sure it is in 'messages' format
            if "user" in data and "assistant" in data:
                normalized = {
                    "messages": [
                        {"role": "user", "content": data["user"]},
                        {"role": "assistant", "content": data["assistant"]}
                    ]
                }
            elif "messages" in data:
                normalized = data
            else:
                print(f"Unknown format, skipping line: {line[:50]}")
                continue

            all_conversations.append(normalized)
        except json.JSONDecodeError as e:
            print(f"Skipping invalid JSON line: {line[:50]} | Error: {e}")

print(f"Loaded {len(all_conversations)} conversations")


Loaded 44 conversations


In [12]:


empathetic_file = "data/raw/empatheticdialogues/train.csv"
output_file = "data/processed/empathetic_conversations.jsonl"

if os.path.exists(empathetic_file):
    try:
        print("📋 Reading CSV file...")
        try:
            df = pd.read_csv(empathetic_file, sep=',', encoding='utf-8', on_bad_lines='skip')
        except Exception:
            print("⚠️ Comma separator failed, trying tab...")
            df = pd.read_csv(empathetic_file, sep='\t', encoding='utf-8', on_bad_lines='skip')

        # Keep only the required columns
        keep_cols = ['conv_id', 'prompt', 'utterance']
        df = df[[col for col in keep_cols if col in df.columns]]

        # Check required columns
        if not {'prompt', 'utterance'}.issubset(df.columns):
            raise ValueError("Missing required columns: 'prompt' and 'utterance'")

        print(f"📊 Columns kept: {list(df.columns)}")
        print(f"📊 Total rows: {len(df)}")

        # If conv_id is missing, create one based on row order
        if 'conv_id' not in df.columns:
            df['conv_id'] = range(len(df))

        # Group by conversation ID if present
        grouped = df.groupby('conv_id')

        for conv_id, group in grouped:
            messages = []

            for _, row in group.iterrows():
                # Add the prompt as user message
                prompt = str(row['prompt']).strip()
                if prompt:
                    messages.append({"role": "user", "content": prompt})

                # Add the utterance as assistant message
                utterance = str(row['utterance']).strip()
                if utterance:
                    messages.append({"role": "assistant", "content": utterance})

            # Only include meaningful conversations
            if len(messages) >= 2:
                all_conversations.append({"messages": messages})

        # Save to JSONL
        os.makedirs(os.path.dirname(output_file), exist_ok=True)
        with open(output_file, 'w', encoding='utf-8') as f:
            for conv in all_conversations:
                f.write(json.dumps(conv, ensure_ascii=False) + '\n')

        print(f"✅ Successfully saved {len(all_conversations)} conversations to {output_file}")

    except Exception as e:
        print(f"⚠️ Error processing EmpatheticDialogues: {e}")
        import traceback
        print(traceback.format_exc())

else:
    print(f"⚠️ EmpatheticDialogues file not found at {empathetic_file}")


📋 Reading CSV file...
📊 Columns kept: ['conv_id', 'prompt', 'utterance']
📊 Total rows: 76668
✅ Successfully saved 17883 conversations to data/processed/empathetic_conversations.jsonl


In [14]:
with open("data/processed/burmese.jsonl", "r", encoding="utf-8") as f:
    for line in f:
        line = line.strip()
        if not line:
            continue
        try:
            data = json.loads(line)
            # Make sure it is in 'messages' format
            if "user" in data and "assistant" in data:
                normalized = {
                    "messages": [
                        {"role": "user", "content": data["user"]},
                        {"role": "assistant", "content": data["assistant"]}
                    ]
                }
            elif "messages" in data:
                normalized = data
            else:
                print(f"Unknown format, skipping line: {line[:50]}")
                continue

            all_conversations.append(normalized)
        except json.JSONDecodeError as e:
            print(f"Skipping invalid JSON line: {line[:50]} | Error: {e}")

print(f"Loaded {len(all_conversations)} conversations")


Loaded 17938 conversations


In [15]:
# Save final Combined Data


output_file = "data/processed/combined3dataset.jsonl"

print(f"\n💾 Saving combined data...")
print(f"📊 Total conversations: {len(all_conversations)}")

with open(output_file, 'w', encoding='utf-8') as f:
    for conv in all_conversations:
        f.write(json.dumps(conv, ensure_ascii=False) + '\n')

print(f"✅ Saved to: {output_file}")


💾 Saving combined data...
📊 Total conversations: 17938
✅ Saved to: data/processed/combined3dataset.jsonl
